#### Import Packages

In [1]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

#### Technology data

In [2]:
n = lk('../nfet_03v3.mat')
p = lk('..//pfet_03v3.mat')

#### Specifications

In [3]:
CL=1e-12; id0 = 10e-6

#### Design choices

In [4]:
gm_id0 = 5
gm_id1 = 16 # power efficient at cost of large gate cap.
gm_id2 = 5  # lower noise from mirror, higher non-dominant pole
l0 = 1
l1 = 0.5
l2 = 1

#### Sizing and benchmarking

In [5]:
# calculate the gm
id1 = id2 = id0/2
gm1 = gm_id1*id1
gm2 = gm_id2*id2

# estimate GBW
GBW = gm1/CL/2/np.pi

# compute DC gain
gm_gds1 = n.lookup('GM_GDS', GM_ID=gm_id1, L=l1)
gds1 = gm1/gm_gds1
gm_gds2 = p.lookup('GM_GDS', GM_ID=gm_id2, L=l2)
gds2 = gm2/gm_gds2
A0 = gm1/(gds1 + gds2)

# size all transistors
jd1 = n.lookup('ID_W', GM_ID=gm_id1, L=l1)
w1 = id1/jd1
jd2 = p.lookup('ID_W', GM_ID=gm_id2, L=l2)
w2 = id1/jd2
w0 = w1       # executive decision

# estimate mirror pole
cgg2 = w2*p.lookup('CGG_W', GM_ID=gm_id2, L=l2)
cdd2 = w2*p.lookup('CDD_W', GM_ID=gm_id2, L=l2)
cdd1 = w2*n.lookup('CDD_W', GM_ID=gm_id1, L=l1)
fp2 = gm2/(2*cgg2+cdd1+cdd2)/(2*np.pi)
CLtot = CL + cdd1 + cdd2

# estimate phase margin (mirror pole, LHP zero, RHP zero)
phip2 = -np.arctan(GBW/fp2)*180/np.pi
fz2 = 2*fp2
phiz2 = +np.arctan(GBW/fz2)*180/np.pi
cgd1 = w1*n.lookup('CGD_W', GM_ID=gm_id1, L=l1)
fz3 = gm1/cgd1/(2*np.pi)
phiz3 = -np.arctan(GBW/fz3)*180/np.pi
PM = 90 + phip2 + phiz2 +phiz3 

df = pd.DataFrame( [id1/1e-6, gm1/1e-6, gm2/1e-6, A0, GBW*1e-6, fp2, PM], \
                   ['id1 (uA)', 'gm1 (uS)', 'gm2 (uS)', 'A0 (V/V)', 'GBW (MHz)', 'fp2 (Hz)', 'PM (deg)'], columns=['Value']); 

pd.set_option('display.float_format', '{:.2e}'.format); df

,Value
id1 (uA),5.00e+00
gm1 (uS),8.00e+01
gm2 (uS),2.50e+01
A0 (V/V),1.53e+02
GBW (MHz),1.27e+01
fp2 (Hz),2.02e+08
PM (deg),8.82e+01


In [6]:
# finger the devices
wfing = 5
nf0 = 1+np.floor_divide(w0, wfing)
nf1 = 1+np.floor_divide(w1, wfing)
nf2 = 1+np.floor_divide(w2, wfing)
df = pd.DataFrame( [(id0*1e6, id1*1e6, id2*1e6), (w0, w1, w2), (l0, l1, l2), (nf0, nf1, nf2)], \
                   ['ID (uA)', 'W (um)', 'L (um)', 'nf'], columns=['M0', 'M1', 'M2']); df.round(2)

,M0,M1,M2
ID (uA),1.00e+01,5.00e+00,5.00e+00
W (um),4.83e+00,4.83e+00,2.25e+00
L (um),1.00e+00,5.00e-01,1.00e+00
nf,1.00e+00,1.00e+00,1.00e+00


#### Lessons from baseline effort

In [7]:
CLtot = CL + cdd1 + cdd2
print(f'CLtot = {CLtot*1e12:0.4f} pF')
print(f'fp2 = {fp2*1e-6:0.4f} MHz')
print(f'fz2 = {fz2*1e-6:0.4f} MHz')
print(f'fz3 = {fz3*1e-6:0.4f} MHz')
print(f'phip2 = {phip2:0.2f} degree')
print(f'phiz2 = {phiz2:0.2f} degree')
print(f'phiz3 = {phiz3:0.2f} degree')


CLtot = 1.0036 pF
fp2 = 202.0059 MHz
fz2 = 404.0117 MHz
fz3 = 19050.1233 MHz
phip2 = -3.61 degree
phiz2 = 1.81 degree
phiz3 = -0.04 degree
